In [1]:
import nltk
from nltk.corpus import stopwords
from nltk import wordpunct_tokenize
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
vectorizer = TfidfVectorizer(max_features = 2000)
naive_bayes_classifier = MultinomialNB()

stopword_english = list(stopwords.words("english"))

with open("Franck-Dernoncourt pubmed-rct master PubMed_200k_RCT/train.txt","r") as f:
    data_train = f.readlines()

with open("Franck-Dernoncourt pubmed-rct master PubMed_200k_RCT/dev.txt","r") as f:
    data_dev = f.readlines()

with open("Franck-Dernoncourt pubmed-rct master PubMed_200k_RCT/test.txt","r") as f:
    data_test = f.readlines()

def clean(sentence, stopword_english= stopword_english):
    sentence = sentence.lower()
#     sentence = re.sub(r'[^\w\s]',"",sentence) ##Removes punc from within words
    words = wordpunct_tokenize(sentence)
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    sentence = [x for x in words if x not in stopword_english]
    sentence = [x for x in sentence if x not in punc] #Removes punc and seperates words
    return " ".join(sentence)

def get_clean_data(sentences):
    labels = []
    clean_sentences = []
    for line in sentences:
        line = line.rstrip("\n")
        a = line.split("\t")
        if len(a)<2:
            continue
        label = a[0]
        sentence = a[1]
        sent= clean(sentence)
        labels.append(label)
        clean_sentences.append(sent)
    return clean_sentences, labels

X_train, Y_train = get_clean_data(data_train)
X_dev, Y_dev = get_clean_data(data_dev)
X_test, Y_test = get_clean_data(data_test)

X_train_tf_idf = vectorizer.fit_transform(X_train)
X_dev_tf_idf = vectorizer.transform(X_dev)
X_test_tf_idf = vectorizer.transform(X_test)

unq, counts = np.unique(Y_train, return_counts=True)
diction = {}
for x in unq:
    diction[x] = len(diction)
reverse_diction = {v:x for x,v in diction.items()}

counts = sum(counts)/counts
counts = counts/np.min(counts)


def numerize(Y, diction= diction):
    return [diction[x] for x in Y]
Y_train_num = numerize(Y_train, diction)
Y_dev_num = numerize(Y_dev, diction)
Y_test_num = numerize(Y_test, diction)




In [2]:
from tensorflow.keras import optimizers, losses, activations, models, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate, SimpleRNN, LSTM

In [3]:
from model_arch import baseline

In [4]:
model = baseline(nclass=5, shape=X_train_tf_idf.shape[-1])

2022-04-09 02:33:50.133590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 02:33:50.162856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 02:33:50.163519: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 02:33:50.185668: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

ResourceExhaustedError: failed to allocate memory [Op:AddV2]

In [6]:
X_train_tf_idf.shape

(2211861, 180672)